In [49]:
import pandas as pd
from langchain_core.prompts import PromptTemplate

In [50]:
from prompts import industry_chain, PROMPT_TEMPLATE

In [51]:
prompt_template = PromptTemplate.from_template(PROMPT_TEMPLATE.lstrip())

In [52]:
prompt_template

PromptTemplate(input_variables=['business_scope', 'corporation_name', 'industry_chain_descriptive_text'], input_types={}, partial_variables={}, template='你是一名产业研究分析师。请依据“企业经营范围”的**明示证据**，判断该企业在给定目标产业链中属于【上游 / 中游 / 下游】的哪一类（可多选）；若缺乏充分证据，请返回“均不属于”。**严禁臆测**。\n\n【判定准则】\n- **若未明确指向该产业链核心环节**，一律不作为归属依据。\n- 如经营范围同时覆盖多个环节，允许多选；但**仅限与目标产业链直接相关**的活动。\n\n【任务】\n1) 从“企业经营范围”中提取与目标产业链直接相关的**证据短语**（原文片段）。\n2) 基于证据，按上述准则判定归属。\n3) 若证据不足或只是笼统表述且未显式对接该产业链核心物料/设备/工艺/产品，返回“均不属于”。\n\n【输入】\n- 企业名: {corporation_name}\n- 企业经营范围: {business_scope}\n- 目标产业链的描述文本: {industry_chain_descriptive_text}\n\n【输出要求】\n- 仅输出**合法 JSON**，不得包含多余文本或注释。\n- 字段含义：\n  - reason：简要给出决策依据（不展开长推理，仅概括要点）。\n  - belong_to：列表，取值仅限 ["上游","中游","下游","均不属于"]（可多选；若选择“均不属于”，则为单一取值）。\n  - evidence：数组，列出用于判定的经营范围**原文片段**（无则给空数组）。\n  - confidence：0~1 的小数，表示总体把握（基于证据充分性与清晰度）。\n\n【输出JSON示例】\n你可以先生成一段思考过程，最后再生成json格式的数据。\n```json\n{{\n  "reason": "经营范围含‘××原料生产’与‘××设备制造’，对应上游；并有‘××系统集成’，对应中游。",\n  "belong_to": ["上游","中游"],\n  "evidence": ["××原料生产", "××设备

In [54]:
# df = pd.read_excel("data/煤化工_上市公司_经纬度_百度API.xlsx")
df = pd.read_excel("data/煤化工_经纬度_百度API.xlsx")

In [55]:
df.shape

(9833, 39)

In [56]:
def set_prompt(chain_name):
    industry_chain_descriptive_text = industry_chain[chain_name]
    def _func(item):
        corporation_name = item["企业名称"]
        business_scope = item["经营范围"]
        prompt = prompt_template.format(
            corporation_name=corporation_name,
            business_scope=business_scope,
            industry_chain_descriptive_text=industry_chain_descriptive_text
        )
        return prompt
    return _func

In [ ]:
# print(set_prompt("煤基聚酯")(df.iloc[0].to_dict()))

In [57]:
prompt_data = []

for k in industry_chain.keys():
    tmp_data = df.apply(set_prompt(k), axis=1).to_list()
    for v in tmp_data:
        prompt_data.append({
            "instruction":"",
            "input":v,
            "output":""
        })

In [58]:
len(prompt_data)

68831

In [59]:
import json

def save_jsonl(obj, output_file):
    with open(output_file, "w") as f:
        for item in obj:
            f.write(json.dumps(item, ensure_ascii=False) + "\n")

In [60]:
# save_jsonl(prompt_data, "data/cisted_companies.jsonl")
save_jsonl(prompt_data, "data/normal_companies.jsonl")